In [28]:
from itertools import chain

from gensim.models import Word2Vec

In [25]:
# load Google's word2vec model

path_to_model = '../../jupyters/vocab_coverage/word2vec_web/GoogleNews-vectors-negative300.bin'
w2v_model = Word2Vec.load_word2vec_format(path_to_model, binary=True)
print('done loading Word2Vec')

done loading Word2Vec


In [26]:
# get word2vec's vocab
word2vec_vocab = w2v_model.vocab.keys()
word2vec_vocab_lower = [item.lower() for item in word2vec_vocab]

In [9]:
def import_tag(datasets=None):
    ''' Imports the datasets into one of two dictionaries

        Dicts:
            train & test

        Keys:
            values >= 4  are "Positive" in both Dictionaries

        '''
    if datasets is not None:
        train = {}
        test = {}
        for k, v in datasets.items(): #gives a list of tuples to iterate
            with open(k) as fpath:
                data = fpath.readlines()
            for val, each_line in enumerate(data):
                if v.endswith("NEG") and v.startswith("TRAIN"):
                    train[val] = each_line
                elif v.endswith("POS") and v.startswith("TRAIN"):
                    train[val + 12500] = each_line
                elif v.endswith("NEG") and v.startswith("TEST"):
                    test[val] = each_line
                else:
                    test[val + 12500] = each_line
        return train, test
    else:
        print('Data not found...')

In [6]:
cd ./../../jupyters/Data/

u'/Users/airwoot/Documents/Anuj/Coding/GitHub/NLP_by_Example/Ch9'

In [10]:
data_locations = {'./../../jupyters/Data/test-neg.txt': 'TEST_NEG',
                  './../../jupyters/Data/test-pos.txt': 'TEST_POS',
                  './../../jupyters/Data/train-neg.txt': 'TRAIN_NEG',
                  './../../jupyters/Data/train-pos.txt': 'TRAIN_POS'}

In [11]:
train, test = import_tag(data_locations)

In [12]:
type(train)

dict

In [22]:
train[0]

'story of a man who has unnatural feelings for a pig starts out with a opening scene that is a terrific example of absurd comedy a formal orchestra audience is turned into an insane violent mob by the crazy chantings of it s singers unfortunately it stays absurd the whole time with no general narrative eventually making it just too off putting even those from the era should be turned off the cryptic dialogue would make shakespeare seem easy to a third grader on a technical level it s better than you might think with some good cinematography by future great vilmos zsigmond future stars sally kirkland and frederic forrest can be seen briefly \n'

In [16]:
def tokenizer(text):
    ''' Simple Parser converting each document to lower-case, then
        removing the breaks for new lines and finally splitting on the
        whitespace
    '''
    text = [document.lower().replace('\n', '').split() for document in text]
    return text

In [17]:
# combine training and test reviews (50K) and tokenize each one of them

print('Loading Data...')
combined = train.values() + test.values()
print('Tokenising...')
combined = tokenizer(combined)

Loading Data...
Tokenising...


In [23]:
print type(combined), len(combined)
print combined[0]

<type 'list'> 50000
['story', 'of', 'a', 'man', 'who', 'has', 'unnatural', 'feelings', 'for', 'a', 'pig', 'starts', 'out', 'with', 'a', 'opening', 'scene', 'that', 'is', 'a', 'terrific', 'example', 'of', 'absurd', 'comedy', 'a', 'formal', 'orchestra', 'audience', 'is', 'turned', 'into', 'an', 'insane', 'violent', 'mob', 'by', 'the', 'crazy', 'chantings', 'of', 'it', 's', 'singers', 'unfortunately', 'it', 'stays', 'absurd', 'the', 'whole', 'time', 'with', 'no', 'general', 'narrative', 'eventually', 'making', 'it', 'just', 'too', 'off', 'putting', 'even', 'those', 'from', 'the', 'era', 'should', 'be', 'turned', 'off', 'the', 'cryptic', 'dialogue', 'would', 'make', 'shakespeare', 'seem', 'easy', 'to', 'a', 'third', 'grader', 'on', 'a', 'technical', 'level', 'it', 's', 'better', 'than', 'you', 'might', 'think', 'with', 'some', 'good', 'cinematography', 'by', 'future', 'great', 'vilmos', 'zsigmond', 'future', 'stars', 'sally', 'kirkland', 'and', 'frederic', 'forrest', 'can', 'be', 'seen', '

In [29]:
#  get the vocab size of w2v_model
len(w2v_model.vocab.keys())

# this is one way to create vocab set from our dataset 
flat_list = chain(*combined)
dataset_vocab = set(flat_list)
len(dataset_vocab)
dataset_vocab_lower = [key.lower() for key in dataset_vocab]

In [ ]:
# This does 2 things - 
# 1) create a small dictionary of word to vectors for words which are only present in our dataset
# 2) Counts the number of words in our corpus not present in Google's word to vec

DIMENSION = 300
zero_vector = np.zeros(DIMENSION)

our_dict = {}
missing_count = 0

for key in dataset_vocab_lower:
    if key in w2v_model:
        our_dict[key] = w2v_model[key]
    else:
        missing_count += 1
        #print("missing key %s" %(key)) 
        our_dict[key] = zero_vector

print len(dataset_vocab_lower)
print missing_count

In [ ]:
def getWordVecs(words):
    vecs = []
    for word in words:
        word = word.replace('\n', '')
        try:
            vecs.append(model[word].reshape((1,300)))
        except KeyError:
            continue
    vecs = np.concatenate(vecs)
    return np.array(vecs, dtype='float')

In [ ]:
with open('food_words.txt', 'r') as infile:
    food_words = infile.readlines()